In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,

    # 아래 주석을 해제하면 console에서 응답이 진행되는 과정을 디버깅 해볼 수 있음 [!주석 해제 후 cell 코드 실행 해야 적용됨]
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

# 셰프 프롬프트
chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

# 셰프 체인 생성
chef_chain = chef_prompt | chat

In [9]:
# 채식 셰프 프롬프트
# 기존의 레시피에서 채식주의자가 먹을 수 있는 재료로 대체할 수 있도록 레시피 생성
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)

# 채식 셰프 체인 생성
veg_chain = veg_chef_prompt | chat

# 셰프 체인과 채식 셰프 체인을 다시 체인으로 연결
# 1. 아래에서 파이널 체인을 호출하면 chef_chain에 "indian" 음식에 대한 레시피를 작성함
# 2. 레시피 생성이 완료되면 "recipe"라는 key안에 chef_chain에서 생성된 레시피 결과물이 담기고
# 3. veg_chain에 "recipe": chef_chain에서 생성된 레시피 형태로 전달되어
# 4. 기존 레시피가 채식 주의자를 위한 레시피로 변경됨
final_chain = {"recipe": chef_chain} | veg_chain

# 체인 호출(실행)
final_chain.invoke({"cuisine": "indian"})



AIMessage(content="As a vegetarian chef specialized in making traditional recipes vegetarian, I can suggest some alternative ingredients for the Indian recipes mentioned:\n\n1. Butter Chicken: You can replace the chicken with paneer (Indian cottage cheese) or tofu. Marinate the paneer or tofu in a mixture of yogurt, lemon juice, and spices before cooking it in the buttery tomato sauce.\n\n2. Chana Masala: To make this chickpea curry even more flavorful, you can add roasted vegetables like bell peppers, zucchini, or sweet potatoes. They will add a nice depth to the dish.\n\n3. Palak Paneer: Instead of paneer, you can use tofu or even boiled and cubed potatoes as a substitute. The spinach gravy will still be rich and creamy with these alternatives.\n\n4. Biryani: For a vegetarian version of biryani, you can use a mix of vegetables like carrots, peas, potatoes, and cauliflower. Make sure to marinate the vegetables in yogurt and spices before layering them with the fragrant rice.\n\n5. Alo